# PREPROCESS IS WORKING 

## converting coords into grch38 has problems

## .bed file has 23k lines while our original data had 18k rows

In [3]:
import pandas as pd
import os
import csv

In [9]:
def read_clash_data(filename="mmc1.txt"):
    """
    Parse Clash data from the given file and write the results to output files.

    Args:
        filename (str): The path to the input file. Default is "data/raw/clash/mmc1.txt".

    Raises:
        FileNotFoundError: If the input file does not exist.

    Returns:
        None.
    """

    with open(filename) as f:
        
        # iter(f) is the same as (line for line in f)
        lines = iter(f)

        columns = []
        data = []
        for line in lines:
            if line.startswith("#"):
                line = line[1:]
                row = next(csv.reader([line], delimiter="\t"))
                columns.append(row)
            else:
                row = next(csv.reader([line], delimiter="\t"))
                data.append(row)

        # removes header text
        columns.pop(0)

        # create the output directory if it doesn"t exist
        output_dir = "."
        os.makedirs(output_dir, exist_ok=True)

        # write the output CSV files
        with open(os.path.join(output_dir, "clash_column_details.csv"), "w") as f1, \
             open(os.path.join(output_dir, "clash_raw.csv"), "w") as f2:
            writer1 = csv.writer(f1, delimiter=",")
            writer2 = csv.writer(f2, delimiter=",")
            writer1.writerows(columns)
            writer2.writerows(data)

## write to csv

In [11]:
read_clash_data()

## processing miRNA and mRNA names and saving to csv

In [12]:
df = pd.read_csv("clash_raw.csv")

# process microRNA_name column
new_cols = df["microRNA_name"].str.split("_", expand=True)
new_cols.columns = ["mirna_accession", "from", "mirna_name", "temp"]
df = pd.concat([df, new_cols], axis=1)
df = df.drop("microRNA_name", axis=1)
df = df.drop("temp", axis=1)
df = df.drop("from", axis=1)

# process mRNA_name column
new_cols = df["mRNA_name"].str.split("_", expand=True)
new_cols.columns = ["ensg", "enst", "gene_name", "temp"]
df = pd.concat([df, new_cols], axis=1)
df = df.drop("mRNA_name", axis=1)
df = df.drop("temp", axis=1)

df.to_csv("clash_raw_renamed_columns.csv")


# parsing clash_raw

In [2]:
def parse_clash_raw(drop_irrelevant_columns=True):

    df = pd.read_csv("clash_raw.csv")

    if drop_irrelevant_columns:

        cols_to_drop = ['miRNA_start', 'miRNA_end','chimeras_decompressed',
                        'experiments', 'experiments_list', 'microRNA_first', 'two_way_merged',
                        "5'UTR", 'CDS', "3'UTR", 'conservation_score',
                        'log2_target_enrichment', 'CLASH_single_reads_ovlp',
                        'CLASH_cluster_ovlp', 'PAR_CLIP_cluster_ovlp']

        df.drop(columns=cols_to_drop, inplace=True)


    # process microRNA_name column
    new_cols = df["microRNA_name"].str.split("_", expand=True)
    new_cols.columns = ["mirna_accession", "from", "mirna_name", "temp"]
    df = pd.concat([df, new_cols], axis=1)
    df = df.drop("microRNA_name", axis=1)
    df = df.drop("temp", axis=1)
    df = df.drop("from", axis=1)

    # process mRNA_name column
    new_cols = df["mRNA_name"].str.split("_", expand=True)
    new_cols.columns = ["ensg", "enst", "gene_name", "temp"]
    df = pd.concat([df, new_cols], axis=1)
    df = df.drop("mRNA_name", axis=1)
    df = df.drop("temp", axis=1)
    
    # dropping rows where len(mrna_sequence) < len(mirna_sequence)
    df = df[df.mRNA_seq_extended.str.len() >= df.miRNA_seq.str.len()]
    
    rename_dict = {
    "seq_ID": "id",
    "miRNA_seq": "mirna_sequence",
    "mRNA_seq_extended": "mrna_sequence",
    "mRNA_start": "true_start",
    "mRNA_end_extended": "true_end",
    "seed_type": "true_seed_type",
    "folding_class": "true_folding_class"
}

    return df.rename(columns=rename_dict)

In [5]:
df = parse_clash_raw(drop_irrelevant_columns=True)
df.to_csv("../../processed/clash/clash_parsed.csv", index=False)


## ENST into genomic coordinates

In [20]:
# transforming ENST accessions into genomic coordinates

import os
from pyensembl import EnsemblRelease

# set pyensembl cache location
os.environ["PYENSEMBL_CACHE_DIR"] = "../../../../data"
# init database
data = EnsemblRelease(60)
# download and index db
data.download()
data.index()

ensts = df["enst"].values.tolist()

chromosomes = []
starts = []
ends = []
strands = []
ensembl_sequences = []


for i in ensts:
    obj = data.transcript_by_id(i)

    chromosomes.append(obj.contig)
    starts.append(obj.start)
    ends.append(obj.end)
    strands.append(obj.strand)
    ensembl_sequences.append(obj.sequence)

df["transcript_chromosome"] = chromosomes
df["transcript_start"] = starts
df["transcript_end"] = ends
df["transcript_strand"] = strands
df["transcript_sequence"] = ensembl_sequences

df.head()

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /run/media/nazif/2F946E411BA61D49/data/pyensembl/GRCh37/ensembl60/Homo_sapiens.GRCh37.60.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /run/media/nazif/2F946E411BA61D49/data/pyensembl/GRCh37/ensembl60/Homo_sapiens.GRCh37.60.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /run/media/nazif/2F946E411BA61D49/data/pyensembl/GRCh37/ensembl60/Homo_sapiens.GRCh37.60.pep.all.fa.gz.pickle


,seq_ID,miRNA_start,miRNA_end,miRNA_seq,mRNA_start,mRNA_end_extended,mRNA_seq_extended,chimeras_decompressed,experiments,experiments_list,...,CLASH_cluster_ovlp,PAR_CLIP_cluster_ovlp,mirna_name,ensg,enst,transcript_chromosome,transcript_start,transcript_end,transcript_strand,transcript_sequence
0,0727A-1038930_1,1,22,TGAGGTAGTAGGTTGTATAGTT,1791,1890,ATTTGTATCTACGATAAAAATTTTTATACAGAACCTACTGCCTCAA...,31,3,"E2,E3,E4",...,NaN,NaN,let-7a,ENSG00000113328,ENST00000340828,5,162864587,162872022,+,AGGGCAGGCGCGGCCCCTTCGGCTCCGAGCTGACCCTGATCAGGGC...
1,L1HS-1112536_1,1,22,TGAGGTAGTAGGTTGTATAGTT,3857,3928,CAGGAAATACCCGTGCAACCAACTACCTCATATTCCATTCAGAATT...,9,2,"E3,E4",...,NaN,1.0,let-7a,ENSG00000100697,ENST00000343455,14,95552566,95623759,-,CGGAGGCGCGGCGCAGGCTGCTGCAGGCCCAGGTGAATGGAGTAAC...
2,L2HS-818542_2,1,22,TGAGGTAGTAGGTTGTATAGTT,2385,2434,ACCCGCTATATGACCTGATGCCTTTCCTTCATTAAAGATGATTCTG...,2,1,E4,...,1.0,1.0,let-7a,ENSG00000080546,ENST00000436639,6,109307640,109416022,-,GATTGCCAGGGCCGCCCTGTGCCCTCTGGCTCGGCGGTGGTGGGCG...
3,L2HS-1161339_2,1,22,TGAGGTAGTAGGTTGTATAGTT,6570,6623,CAATGACTATGCAACCATACCTTACCACTAAATGTAGTACGCAAAA...,2,1,E4,...,NaN,NaN,let-7a,ENSG00000164190,ENST00000282516,5,36876861,37066515,+,TCCGGTCGGCATTTTGTTCTGAGAGGGAGAGACGGAACGAGAGAGA...
4,L2-407944_2,1,22,TGAGGTAGTAGGTTGTATAGTT,1164,1208,AATACTGGGAAACCTGCTACTTCGTCAGCTAACCAGAAACCTGTG,2,1,E4,...,NaN,NaN,let-7a,ENSG00000138785,ENST00000340139,4,106603784,106629838,-,AGGGACCACCGGGAACAGACGGATCGGCAGGGCGGGGCGGAACGGT...


In [6]:
df.transcript_chromosome.value_counts()

1                  2055
17                 1522
19                 1291
12                 1247
16                 1019
2                   977
6                   923
11                  844
7                   841
3                   814
5                   803
10                  729
X                   728
9                   642
22                  579
15                  544
8                   517
14                  441
20                  434
4                   420
13                  258
21                  250
MT                  213
18                  191
HSCHR6_MHC_DBB      102
HSCHR6_MHC_COX       52
HSCHR6_MHC_APD       18
HSCHR6_MHC_MANN      16
HSCHR6_MHC_MCF       15
HSCHR6_MHC_SSTO      13
HSCHR6_MHC_QBL       11
Y                     4
HSCHR17_1             1
Name: transcript_chromosome, dtype: int64

## generating .bed file to convert old (ENSEMBL60) coordinates into most recent

In [8]:
# uncomment if you want to generate the bed file
# from fuc import pybed

# # filter & rename columns for pybed
# coord_df = df[["chromosome", "start", "end"]]
# coord_df.columns = ["Chromosome", "Start", "End"]

# # call pybed to turn into .bed file
# bf = pybed.BedFrame.from_frame(meta=[], data=coord_df)
# bf.to_file("clash_grch37_coordinates.bed")

## importing new coordinate data

In [8]:
# drop old coords
df.drop(["transcript_chromosome", "transcript_start", "transcript_end"], axis=1, inplace=True)


# read & merge new ones
new_coords = pd.read_csv("../../external/clash/clash_grch38_coordinates.bed", sep="\t", header=None)
new_coords.columns = ["chromosome", "start", "end"]

df = pd.concat([df, new_coords], axis=1)

In [24]:
new_coords = pd.read_csv("../../external/clash/clash_grch38_coordinates.bed", sep="\t", header=None)
new_coords.columns = ["chromosome", "start", "end"]

new_coords.chromosome.value_counts()

1                           3317
X                           1810
17                          1717
19                          1494
7                           1418
12                          1304
16                          1025
2                           1022
11                           955
6                            949
3                            937
22                           926
5                            827
10                           759
8                            671
9                            659
15                           549
20                           548
21                           533
14                           530
4                            425
13                           264
MT                           213
18                           209
CHR_HSCHR6_MHC_DBB_CTG1      102
CHR_HSCHR6_MHC_COX_CTG1       52
CHR_HSCHR6_MHC_APD_CTG1       18
CHR_HSCHR6_MHC_MANN_CTG1      16
CHR_HSCHR6_MHC_MCF_CTG1       15
CHR_HSCHR6_MHC_SSTO_CTG1      13
CHR_HSCHR6

CLASH df original length: 18k

len after transforming coordinates via ENSEMBL: 23k

we shouldn"t transform the data?

In [11]:
df.tail()

,seq_ID,miRNA_start,miRNA_end,miRNA_seq,mRNA_start,mRNA_end_extended,mRNA_seq_extended,chimeras_decompressed,experiments,experiments_list,...,CLASH_cluster_ovlp,PAR_CLIP_cluster_ovlp,mirna_name,ensg,enst,strand,ensembl_sequence,chromosome,start,end
23289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,2841486,2866956
23290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12904831,12910913
23291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12910913,12920478
23292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12904831,12910913
23293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12910913,12920478


In [12]:
import genomepy
a = genomepy.Annotation("GRCh38")

15:26:05 | WARNING | Could not find 'GRCh38.annotation.bed(.gz)' in directory /home/nazif/.local/share/genomes/GRCh38. Methods using this file won't work!
15:26:05 | WARNING | Could not find 'GRCh38.annotation.gtf(.gz)' in directory /home/nazif/.local/share/genomes/GRCh38. Methods using this file won't work!
